# DSFB IEEE L-CSS High-Rate Estimation Trust Analysis Figures

This notebook generates publication-ready figures for the IEEE L-CSS (Letters of Control Systems Society) submission on DSFB high-rate estimation trust analysis.

## Reproduce benchmark outputs first

```bash
cargo run --release --manifest-path crates/dsfb-lcss-hret/Cargo.toml -- --run-default
cargo run --release --manifest-path crates/dsfb-lcss-hret/Cargo.toml -- --run-sweep
```

Expected files in a timestamped run directory:
- `output-dsfb-lcss-hret/<timestamp>/summary.csv`
- `output-dsfb-lcss-hret/<timestamp>/heatmap.csv`
- `output-dsfb-lcss-hret/<timestamp>/trajectories.csv`

## Usage in Google Colab

1. Upload the generated CSV files from your local machine
2. Or modify the `DATA_DIR` variable to point to your data location
3. Run all cells to generate figures
4. Download the figures from the `figures/` directory

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

# IEEE L-CSS figure style settings
FIG_DIR = Path("figures")
FIG_DIR.mkdir(parents=True, exist_ok=True)

# IEEE standard figure width for single column (3.5 inches)
IEEE_SINGLE_COL_WIDTH = 3.5
IEEE_DOUBLE_COL_WIDTH = 7.16

plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 8,
    "font.family": "serif",
    "font.serif": ["Times New Roman"],
    "text.usetex": False,  # Set to True if LaTeX is available
    "axes.labelsize": 8,
    "axes.titlesize": 9,
    "xtick.labelsize": 7,
    "ytick.labelsize": 7,
    "legend.fontsize": 7,
    "axes.grid": True,
    "grid.alpha": 0.3,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "figure.figsize": (IEEE_SINGLE_COL_WIDTH, 2.5),
})

print("Setup complete. Figure output directory:", FIG_DIR.absolute())

In [ ]:
# Data directory configuration
# Modify this to point to your data directory
BASE_DIR = Path("output-dsfb-lcss-hret")

# Find the latest timestamped directory
def find_latest_run_dir(base_dir):
    if not base_dir.exists():
        print(f"Base directory {base_dir} does not exist.")
        print("Please upload your data files or adjust BASE_DIR.")
        return None
    
    # Look for timestamped subdirectories
    subdirs = [d for d in base_dir.iterdir() if d.is_dir()]
    if not subdirs:
        # Check if CSV files are directly in base_dir
        if (base_dir / "summary.csv").exists():
            return base_dir
        print(f"No subdirectories found in {base_dir}")
        return None
    
    # Sort by name (timestamp format YYYYMMDD-HHMMSS ensures chronological order)
    subdirs.sort(reverse=True)
    for subdir in subdirs:
        if (subdir / "summary.csv").exists():
            return subdir
    
    print(f"No valid run directories found in {base_dir}")
    return None

DATA_DIR = find_latest_run_dir(BASE_DIR)

if DATA_DIR is None:
    print("\n⚠️  Data directory not found!")
    print("Please:")
    print("  1. Run the benchmark first, or")
    print("  2. Upload CSV files to Colab, or")
    print("  3. Set DATA_DIR manually to your data location")
else:
    print(f"✓ Using data directory: {DATA_DIR}")
    print(f"  Files found:")
    for csv_file in DATA_DIR.glob("*.csv"):
        print(f"    - {csv_file.name}")

## Figure 1: Method Comparison

In [ ]:
if DATA_DIR is not None:
    # Load summary data
    summary_df = pd.read_csv(DATA_DIR / "summary.csv")
    print("Summary data:")
    print(summary_df)
    print()
    
    # Create bar plot comparing methods
    fig, ax = plt.subplots(figsize=(IEEE_SINGLE_COL_WIDTH, 2.5))
    
    x = np.arange(len(summary_df))
    width = 0.6
    
    bars = ax.bar(x, summary_df['rmse_mean'], width, 
                   yerr=summary_df['rmse_std'],
                   capsize=3,
                   color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:len(summary_df)],
                   alpha=0.8)
    
    ax.set_xlabel('Estimation Method')
    ax.set_ylabel('RMSE')
    ax.set_title('Estimation Method Performance Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(summary_df['method'].str.upper())
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    fig_path = FIG_DIR / "fig1_method_comparison.pdf"
    plt.savefig(fig_path, bbox_inches='tight', dpi=300)
    plt.savefig(FIG_DIR / "fig1_method_comparison.png", bbox_inches='tight', dpi=300)
    print(f"✓ Saved: {fig_path}")
    plt.show()
else:
    print("⚠️  Skipping: Data directory not found")

## Figure 2: Trajectory Comparison

In [ ]:
if DATA_DIR is not None and (DATA_DIR / "trajectories.csv").exists():
    # Load trajectory data
    traj_df = pd.read_csv(DATA_DIR / "trajectories.csv")
    print(f"Trajectory data: {len(traj_df)} time steps")
    print(traj_df.head())
    print()
    
    # Create trajectory plot
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(IEEE_SINGLE_COL_WIDTH, 4.0), sharex=True)
    
    # Plot 1: True vs Estimated
    ax1.plot(traj_df['time'], traj_df['true_x'], 'k-', linewidth=1.5, label='True', alpha=0.7)
    ax1.plot(traj_df['time'], traj_df['est_x'], 'b--', linewidth=1.0, label='Estimated', alpha=0.8)
    ax1.set_ylabel('State $x$')
    ax1.set_title('State Estimation Trajectory')
    ax1.legend(loc='best', framealpha=0.9)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Error
    ax2.plot(traj_df['time'], traj_df['error'], 'r-', linewidth=1.0, alpha=0.7)
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Estimation Error')
    ax2.set_title('Absolute Estimation Error')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    fig_path = FIG_DIR / "fig2_trajectory.pdf"
    plt.savefig(fig_path, bbox_inches='tight', dpi=300)
    plt.savefig(FIG_DIR / "fig2_trajectory.png", bbox_inches='tight', dpi=300)
    print(f"✓ Saved: {fig_path}")
    plt.show()
else:
    print("⚠️  Skipping: Trajectory data not found")

## Figure 3: Parameter Sweep Heatmap

In [ ]:
if DATA_DIR is not None and (DATA_DIR / "heatmap.csv").exists():
    # Load heatmap data
    heatmap_df = pd.read_csv(DATA_DIR / "heatmap.csv")
    print(f"Heatmap data: {len(heatmap_df)} parameter combinations")
    print(heatmap_df.head())
    print()
    
    # Pivot for heatmap
    heatmap_pivot = heatmap_df.pivot(index='param2', columns='param1', values='rmse')
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(IEEE_SINGLE_COL_WIDTH, 2.8))
    
    im = ax.imshow(heatmap_pivot, aspect='auto', cmap='viridis', origin='lower')
    
    # Set ticks
    ax.set_xticks(np.arange(len(heatmap_pivot.columns)))
    ax.set_yticks(np.arange(len(heatmap_pivot.index)))
    ax.set_xticklabels([f"{x:.2f}" for x in heatmap_pivot.columns])
    ax.set_yticklabels([f"{y:.2f}" for y in heatmap_pivot.index])
    
    # Rotate the tick labels for better readability
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    ax.set_xlabel('Parameter 1')
    ax.set_ylabel('Parameter 2')
    ax.set_title('RMSE Parameter Sweep')
    
    # Add colorbar
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('RMSE', rotation=270, labelpad=15)
    
    plt.tight_layout()
    fig_path = FIG_DIR / "fig3_parameter_sweep.pdf"
    plt.savefig(fig_path, bbox_inches='tight', dpi=300)
    plt.savefig(FIG_DIR / "fig3_parameter_sweep.png", bbox_inches='tight', dpi=300)
    print(f"✓ Saved: {fig_path}")
    plt.show()
else:
    print("⚠️  Skipping: Heatmap data not found")

## Summary

All figures have been generated and saved to the `figures/` directory:
- `fig1_method_comparison.pdf` (and .png)
- `fig2_trajectory.pdf` (and .png)
- `fig3_parameter_sweep.pdf` (and .png)

These figures are formatted for IEEE L-CSS submission requirements.

In [ ]:
# List all generated figures
print("Generated figures:")
for fig_file in sorted(FIG_DIR.glob("*.pdf")):
    print(f"  - {fig_file.name}")
for fig_file in sorted(FIG_DIR.glob("*.png")):
    print(f"  - {fig_file.name}")